<a href="https://colab.research.google.com/github/LHW001abc/seq2seq-using-LSTM-machine-translation-Arabic-to-English/blob/main/seq2seq_using_LSTM_%7C_machine_translation_Arabic_to_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as  np
import keras
import os
from pathlib import Path


In [2]:
batch_size = 64 #batch size of training
epochs = 100   # Number of epochs to train for

latent_dim = 256 # latent dimensonality of the encoding space
num_samples = 10000 # Number of samples to train on .



data_path = "/content/ara.txt"



In [3]:
with open(data_path , 'r') as f  :
  lines = f.read().split('\n')
  print(len(lines))
  for i in range(5):
    print(lines[i])


12524
Hi.	مرحبًا.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #629296 (Samer)
Run!	اركض!	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #1245450 (saeb)
Duck!	اخفض رأسك!	CC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036391 (KeEichi)
Duck!	اخفضي رأسك!	CC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036392 (KeEichi)
Duck!	اخفضوا رؤوسكم!	CC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #9036393 (KeEichi)


In [4]:
with open (data_path , 'r') as f :
  lines = f.read().split('\n')
for line in lines[: min(num_samples , len(lines) -1)]:
    input_text , target_text , _ =   line.split('\t')

    target_text = '\t' + target_text + '\n'

    print(target_text)



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
	إنه يتحدث على الهاتف.

	خطابها أثَر في الجمهور.

	هذه السلة مليئة بالفواكه

	والدتهُ كانت تخجل منهُ.

	كيف تجرأ على شتم أختي!

	كيف تجرء على قول شيءٍ كهذا؟

	كم كلّفت التذاكر؟

	كيف سندفع ديوننا الآن؟

	كيفَ سنسدد ديوننا الآن؟

	أسرع و إلا تأخرت.

	أنا معجبةٌ بمهارته في القيادة.

	يعجبني فيك شجاعتك.

	أنا طالب في هذه المدرسة.

	أنا سعيد بسماع صوتك.

	أقدر تعاونك معنا.

	أتيت هنا آملا رؤية توم.

	لا أستطيع أن أدع ذلك يحدث مجددا.

	لا أثق بأمثاله.

	لقد سببت له الكثير من المتاعب.

	فضلت أن أنظر إلى الجهة الأخرى.

	إخترت أن أنظر للجهة الأخرى.

	لم أعره انتباها

	لا اهتم بما يقوله الآخرون.

	لا أعتبر توم صديقا لي.

	لا أعرف كيف أشكرك.

	لا أحب هذه المدينة على الإطلاق.

	لا تعجبني هذه الفكرة البتة.

	لا أعتقد أن توم هو المخطئ.

	لا أريد رؤيتك مرّة ثانية

	أنا أنتظر قرارك على أحر من الجمر.

	استأذنت لدقيقة.

	أعطيت أختي قاموساً.

	أنسجم جيدًا مع زملائي في العمل

	أخذت قيلولة لمدة ساعة.

	لدي ألم في أكتافي.

	لدي ب

In [10]:
#vectorize the data

#define the list and the dictionary
input_texts  = []
target_texts = []
input_chars = set()
target_chars = set()

with open (data_path , 'r') as f :
  lines = f.read().split('\n')
for line in lines[: min(num_samples , len(lines) -1)]:
    input_text , target_text , _ =   line.split('\t')

    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
      if char not in input_chars:
        input_chars.add(char)
    for char in target_text:
      if char not in target_chars:
        target_chars.add(char)

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))

num_encoder_tokens = len(input_chars) # unique car in eng
num_decoder_tokens = len(target_chars) # unique car in ara

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_text))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 36
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 36
Max sequence length for outputs: 54


In [8]:
target_text

'\tنصحتْه ألّا يأكل كثيراً.\n'

In [14]:
input_texts

['Hi.',
 'Run!',
 'Duck!',
 'Duck!',
 'Duck!',
 'Help!',
 'Jump!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello.',
 'Hello.',
 'Hurry!',
 'Hurry!',
 'I see.',
 'I won!',
 'Relax.',
 'Smile.',
 'Cheers!',
 'Got it?',
 'He ran.',
 'I know.',
 'I know.',
 'I know.',
 "I'm 19.",
 "I'm OK.",
 'Listen.',
 'No way!',
 'Really?',
 'Thanks!',
 'Thanks.',
 'Why me?',
 'Awesome!',
 'Be cool.',
 'Beat it.',
 'Call me.',
 'Call me.',
 'Come in.',
 'Come in.',
 'Come on!',
 'Come on!',
 'Come on!',
 'Get out!',
 'Get out!',
 'Get out.',
 'Go away.',
 'Go away.',
 'Go away.',
 'Goodbye!',
 'Goodbye!',
 'Hang on.',
 'He came.',
 'He runs.',
 'Help me!',
 'Help me.',
 'Help me.',
 'Hold on.',
 'I agree.',
 "I'm sad.",
 'Me, too.',
 'Me, too.',
 'Shut up!',
 'Shut up!',
 'Shut up!',
 'Shut up!',
 'Stop it.',
 'Take it.',
 'Tell me.',
 'Tom won.',
 'Tom won.',
 'Wake up!',
 'Welcome.',
 'Welcome.',
 'Welcome.',
 'Welcome.',
 'Who won?',
 'Who won?',
 'Why not?',
 'Why not?',
 'H

In [11]:
input_chars

[' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '€']

In [13]:
[len(txt) for txt in input_texts]

[3,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 

In [16]:
#encoding
input_token_index = dict([(char , i) for i , char in enumerate(input_chars)])
target_token_index = dict([(char , i) for i , char in enumerate(target_chars)])


In [ ]:
input_token_index

In [28]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
#num_encoder_tocken = unique car in eng
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [21]:
encoder_input_data.shape

(10000, 36, 71)

In [22]:
input_token_index['j']

53

In [26]:
#one hot encoding
for  i , (input_text , target_text) in enumerate(zip(input_texts , target_texts)):
  for t , char in enumerate(input_text):
    encoder_input_data[i , t , input_token_index[char]] = 1.0
  # The following line is attempting to find the index for an empty string, which is not present in the dictionary.
  # Replace '' with a space ' ' to pad with spaces instead of empty strings.
  encoder_input_data[i, t+1 :, input_token_index[' ']] = 1.0

In [30]:
for  i , (input_text , target_text) in enumerate(zip(input_texts , target_texts)):
  for t , char in enumerate(target_text):
    decoder_input_data[i , t , target_token_index[char]] = 1.0
    if t > 0 :
      decoder_target_data[i , t-1 , target_token_index[char]] = 1.0
      # The following line is attempting to find the index for an empty string, which is not present in the dictionary.
  decoder_input_data[i, t+1 :, target_token_index[' ']] = 1.0
  decoder_target_data[i, t:, target_token_index[' ']] = 1.0


In [31]:
#define an input sequence and process it :
encoder_inputs = keras.Input(shape=(None , num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim , return_state = True)
encoder_outputs , state_h , state_c = encoder(encoder_inputs)

encoder_states = [state_h , state_c]

decoder_inputs = keras.Input(shape=(None , num_decoder_tokens))

decoder_lstm = keras.layers.LSTM(latent_dim , return_sequences = True , return_state = True)
decoder_outputs , _ , _ = decoder_lstm(decoder_inputs , initial_state = encoder_states)
deocder_dense  = keras.layers.Dense(num_decoder_tokens , activation = 'softmax')
decoder_outputs = deocder_dense(decoder_outputs)

model = keras.Model([encoder_inputs , decoder_inputs] , decoder_outputs)

In [32]:
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [33]:
model.fit(
    [encoder_input_data , decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 8s 23ms/step - loss: 1.3961 - accuracy: 0.7008 - val_loss: 1.7670 - val_accuracy: 0.5842
Epoch 2/100
125/125 [==============================] - 2s 15ms/step - loss: 1.1256 - accuracy: 0.7142 - val_loss: 1.5841 - val_accuracy: 0.5843
Epoch 3/100
125/125 [==============================] - 1s 11ms/step - loss: 1.0441 - accuracy: 0.7270 - val_loss: 1.4723 - val_accuracy: 0.6100
Epoch 4/100
125/125 [==============================] - 1s 12ms/step - loss: 0.9819 - accuracy: 0.7424 - val_loss: 1.4402 - val_accuracy: 0.6135
Epoch 5/100
125/125 [==============================] - 1s 11ms/step - loss: 0.9325 - accuracy: 0.7538 - val_loss: 1.3398 - val_accuracy: 0.6454
Epoch 6/100
125/125 [==============================] - 2s 12ms/step - loss: 0.8956 - accuracy: 0.7623 - val_loss: 1.3056 - val_accuracy: 0.6509
Epoch 7/100
125/125 [==============================] - 1s 11ms/step - loss: 0.8709 - accuracy: 0.7677 - val_loss: 1.2685 - val_accuracy:

In [34]:
#save the model
model.save('s2s')